<a href="https://colab.research.google.com/github/tardigrades2/jupyter-notebook/blob/master/ssd_mobilenet_v1_0_75_depth_quantized_300x300_pets_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Change model name and config file name

In [0]:
# Pre-train model name to download
%env MODEL_NAME=ssd_mobilenet_v1_0.75_depth_quantized_300x300_coco14_sync_2018_07_18

In [0]:
# Copy sample config file from model zoo, you can create a file for yourself
%env CONFIG_FILE=ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync

# 2.Install prerequisites
Follow: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md


In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

In [0]:
cd /content/models/research

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

## Test the API

In [0]:
!python object_detection/builders/model_builder_test.py

# 3.Prepare input data

## Get the oxford pets dataset

In [0]:
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xvf images.tar.gz
!tar -xvf annotations.tar.gz

## Build tfrecords

In [0]:
!python object_detection/dataset_tools/create_pet_tf_record.py \
    --label_map_path=object_detection/data/pet_label_map.pbtxt \
    --data_dir=`pwd` \
    --output_dir=`pwd`
!ls *.record*

# 4.Prepare model and config

## Get the pre-trained model for transfer learning

In [0]:
!wget http://download.tensorflow.org/models/object_detection/${MODEL_NAME}.tar.gz
!tar -xvf ${MODEL_NAME}.tar.gz
!cp ${MODEL_NAME}/model.ckpt.* .

## Get the config file and edit it to have the right directories

In [0]:
!cp object_detection/samples/configs/${CONFIG_FILE}.config .
# Change config file content
!sed -i "s|PATH_TO_BE_CONFIGURED|/content/models/research|g" ${CONFIG_FILE}.config
!sed -i "s|/content/models/research/pet_label_map.pbtxt|/content/models/research/object_detection/data/pet_label_map.pbtxt|g" ${CONFIG_FILE}.config
!sed -i "s|/content/models/research/pet_train.record|/content/models/research/pet_faces_train.record|g" ${CONFIG_FILE}.config
!sed -i "s|/content/models/research/pet_val.record|/content/models/research/pet_faces_val.record|g" ${CONFIG_FILE}.config
!sed -i "s|128|28|g" ${CONFIG_FILE}.config

# 5.Run Tensorboard via ngrok

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

In [0]:
get_ipython().system_raw('tensorboard --logdir /content/models/research --host 0.0.0.0 --port 6006 &')


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# 6.Train the model

In [0]:
!mkdir train

In [0]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path=/content/models/research/${CONFIG_FILE}.config \
    --model_dir=/content/models/research/train \
    --alsologtostderr \
    --num_train_steps=200000 \
    --num_eval_steps=1000

# 7.Download model

## Convert last checkpoint into a model


In [0]:
!rm -r  exported_graphs; mkdir exported_graphs

In [0]:
%env CHECKPOINT_NUMBER=12526

In [0]:
!python object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path ${CONFIG_FILE}.config \
    --trained_checkpoint_prefix train/model.ckpt-${CHECKPOINT_NUMBER} \
    --output_directory exported_graphs

## Upload to GG Drive

### Compress the graph

In [0]:
!tar -zcvf ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.tar.gz exported_graphs

### Prepare for upload to GG Drive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Upload the file into GG Drive

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.tar.gz'})
uploaded.SetContentFile('ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.tar.gz')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 8*.Continue training in other session 
(run only if you want to continue train current in next session)

## Zip train folder to upload

In [0]:
!tar -zcvf train_ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.tar.gz train

## Upload to Drive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'train_ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.tar.gz'})
uploaded.SetContentFile('train_ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.tar.gz')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 9*.Get pre-session train folder from GG Drive
(not run when this is first session of Colab)

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
!cp "/gdrive/My Drive/train.tar.gz" "train.tar.gz"

In [0]:
!tar -xvf train.tar.gz

# Utility cells

## Monitor the loss convergence without tensorboard

In [0]:
import tensorflow as tf
import re


for event in tf.train.summary_iterator('train/events.out.tfevents.1525935471.f00cf4d2a1b5'):
    for value in event.summary.value:
        if value.tag == 'Losses/Loss/RPNLoss/objectness_loss':
            if value.HasField('simple_value'):
                print(value.simple_value)

## Show how long you have use this session

In [0]:
!uptime